# PROJECT: ANPR/ALPR (Auto Number/License Plate Recognition)

In [1]:
from ultralytics import YOLO
import cv2
import easyocr
import matplotlib.pyplot as plt
import numpy as np
from paddleocr import PaddleOCR,draw_ocr

c:\Program Files\Python39\lib\site-packages\paddle\base\framework.py:688: UserWarning: You are using GPU version Paddle, but your CUDA device is not set properly. CPU device will be used by default.
  warnings.warn(


In [2]:
ocr= PaddleOCR(use_angle_cls=True,lang="en")

[2024/10/15 23:14:30] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='C:\\Users\\Shorya Sharma/.paddleocr/whl\\det\\en\\en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='C:\\Users\\Shorya Sharma/.paddleocr/whl\\rec\\en\\en_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320'

In [3]:
model= YOLO("ANPR.pt")

# Using Video:

In [13]:
cap= cv2.VideoCapture("Traffic Control CCTV.mp4")

while True:

    ret,frame= cap.read()
    
    frame= cv2.resize(frame,(1200,700))
    
    x1, y1 = 50, 400  # Top-left corner of the ROI
    x2, y2 = 1150, 680  # Bottom-right corner of the ROI
    
    roi = frame[y1:y2, x1:x2] # ROI Frame for detection
    
    cv2.rectangle(frame,pt1=(50,400),pt2=(1150,680),color=(255,0,0),thickness=2) # ROI
    
    results= model(roi) # Detect License Plate in ROI 

    for r in results:
        if r.boxes.xyxy.shape[0] is not None and r.boxes.xyxy.shape[0] > 0:
            for box in r.boxes.xyxy:
                x1, y1, x2, y2 = box  # Extract the coordinates of the detected license plate

                cv2.rectangle(roi,pt1=(int(x1),int(y1)),pt2=(int(x2),int(y2)),color=(0,255,0),thickness=2) # Draw rect for detected plate
                
                plate_image= roi[int(y1):int(y2),int(x1):int(x2)] # crop detected plate for recognition
                
                input_plate= cv2.resize(plate_image,(300,150))
                
                cv2.imshow("input_plate",input_plate) # To show detected plate image
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break

                results= ocr.ocr(plate_image,cls=True) # Put image in Paddleocr for recognition
                
                for line in results: # Extract results of recognition
                    
                    if line is not None and len(line)>0:
                        
                        text, confidence = line[0][1][0], line[0][1][1]  # Extract text and confidence score
            
                        if confidence >= 0.90:
                            print(f"License Plate No.: {text} (Confidence: {confidence})")
                        
                        if line:
                            cv2.putText(roi,text=f"License:{line[0][1][0]}",org=(int(x1)-10,int(y1)-10),fontFace=cv2.FONT_HERSHEY_TRIPLEX,fontScale=0.5,color=(255,0,0),thickness=1)
                        else:
                            cv2.putText(roi, text="Not recognized", org=(int(x1)-10,int(y1)-10), fontFace=cv2.FONT_HERSHEY_TRIPLEX, fontScale=0.5, color=(255,0,0), thickness=1)
            
    cv2.imshow("output",frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
            
cap.release()
cv2.destroyAllWindows()
    


0: 192x640 2 License Plates, 43.2ms
Speed: 2.1ms preprocess, 43.2ms inference, 0.0ms postprocess per image at shape (1, 3, 192, 640)
[2024/10/15 23:28:46] ppocr DEBUG: dt_boxes num : 1, elapsed : 0.028839826583862305
[2024/10/15 23:28:46] ppocr DEBUG: cls num  : 1, elapsed : 0.014282464981079102
[2024/10/15 23:28:47] ppocr DEBUG: rec_res num  : 1, elapsed : 0.04903745651245117
[2024/10/15 23:28:47] ppocr DEBUG: dt_boxes num : 1, elapsed : 0.01890087127685547
[2024/10/15 23:28:47] ppocr DEBUG: cls num  : 1, elapsed : 0.006507396697998047
[2024/10/15 23:28:47] ppocr DEBUG: rec_res num  : 1, elapsed : 0.07198405265808105

0: 192x640 2 License Plates, 44.3ms
Speed: 0.0ms preprocess, 44.3ms inference, 0.0ms postprocess per image at shape (1, 3, 192, 640)
[2024/10/15 23:28:47] ppocr DEBUG: dt_boxes num : 1, elapsed : 0.015636920928955078
[2024/10/15 23:28:47] ppocr DEBUG: cls num  : 1, elapsed : 0.015625476837158203
[2024/10/15 23:28:47] ppocr DEBUG: rec_res num  : 1, elapsed : 0.0637691020

# Real Time ANPR:

In [14]:
# Initialize video capture from the camera (0 is usually the default camera)
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    
    if not ret:
        print("Failed to grab frame")
        break  # Exit loop if camera fails to read frame

    frame = cv2.resize(frame, (1200, 700))

    # Detect License Plates in the entire frame
    results = model(frame)  # Remove the ROI

    if results and results[0].boxes.xyxy.shape[0] > 0:
        for box in results[0].boxes.xyxy:
            x1, y1, x2, y2 = map(int, box)  # Extract coordinates

            # Draw rectangle for detected plate
            cv2.rectangle(frame, pt1=(x1, y1), pt2=(x2, y2), color=(0, 255, 0), thickness=2)

            plate_image = frame[y1:y2, x1:x2]  # Crop detected plate for recognition

            # Resize for uniformity
            input_plate = cv2.resize(plate_image, (300, 150))
            
            # Perform OCR on the cropped plate image
            results_ocr = ocr.ocr(plate_image, cls=True)

            # Display the detected plate image briefly
            cv2.imshow("input_plate", input_plate)
            cv2.waitKey(1)  # Show for a very short time
            
            for line in results_ocr:  # Extract results of recognition
                if line is not None and len(line) > 0:
                    text, confidence = line[0][1][0], line[0][1][1]  # Extract text and confidence score
                    
                    if confidence >= 0.90:
                        print(f"License Plate No.: {text} (Confidence: {confidence})")
                        cv2.putText(frame, text=f"License: {text}", org=(x1, y1 - 10), 
                                    fontFace=cv2.FONT_HERSHEY_TRIPLEX, fontScale=0.5, 
                                    color=(255, 0, 0), thickness=1)
                    else:
                        cv2.putText(frame, text="Not recognized", org=(x1, y1 - 10), 
                                    fontFace=cv2.FONT_HERSHEY_TRIPLEX, fontScale=0.5, 
                                    color=(255, 0, 0), thickness=1)

    cv2.imshow("output", frame)  # Show the output frame
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()



0: 384x640 (no detections), 160.4ms
Speed: 15.0ms preprocess, 160.4ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 118.7ms
Speed: 3.0ms preprocess, 118.7ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 103.6ms
Speed: 7.9ms preprocess, 103.6ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 93.6ms
Speed: 5.9ms preprocess, 93.6ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 92.3ms
Speed: 9.4ms preprocess, 92.3ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 90.6ms
Speed: 3.0ms preprocess, 90.6ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 77.0ms
Speed: 3.6ms preprocess, 77.0ms inference, 5.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 71.5ms
Speed: 3.3ms preprocess, 7

In [33]:
line

In [26]:
line[0][1][0]

'B-98ED2'

In [27]:
line[0][1][1]

0.8796282410621643